In [1]:
import math
import pandas as pd
import plotly
import plotly.express as px
from matplotlib import pyplot as plt

from cadCAD.configuration.utils import config_sim

from cadCAD.engine import ExecutionMode, ExecutionContext
from cadCAD.engine import Executor

In [2]:
import networkx as nx   
import numpy as np
from cadCAD.configuration import Experiment
from numpy import random
from cadCAD import configs
del configs[:] # Clear any prior configs

experiment = Experiment()

# Setup / Preparatory Steps

## Initializing Agents Network

<center>
<img src="./images/FinTech.jpg"
     alt="Diff spec"
     style="width: 100%" />
</center>

<center>
<img src="./images/FinTech_CL.jpg"
     alt="Diff spec"
     style="width: 100%" />
</center>

In [3]:
initial_state = {
    'population': 100, # number of agents
    'treasury': 0, # in usd
    'expenses': 0, # in usd
    'net_income': 0,
    'external_kyc': 0,
    'external_exchange':0,
    'external_liquidity':0,
    'external_fiat':0,
    'external_wallet':0,
}

system_params = {
    'increase_rate': [0.17, 0.22],
    'decrease_rate': [0.1, 0.12],
    'fees': [0.01, 0.007],
    'buy_rate': [0.25, 0.15],
    'sell_rate': [0.30, 0.20],
    'expenses_kyc': [500.0],
    'expenses_exchange_provider': [0.00015],
    'expenses_liquidity_providers': [0.001],
    'expenses_fiat_gateway': [0.025], 
    'expenses_wallet_provider': [1000.0]
}

## 1. Logic


\begin{align}
\large population_t &\large= population_{t-1} + {\Delta population} \quad \textrm{(users)} \tag{1} \\
\large treasury_t &\large= treasury_{t-1} + {\Delta treasury} * fees \quad \textrm{(income from fees)} \tag{2}
\end{align}

where the rate of change ($\Delta$) is:
\begin{align}
\large {\Delta population} &\large= \alpha * population_{t-1} - \epsilon * population_{t-1} \quad \textrm{(new users/month)} \\
\large {\Delta treasury} &\large= \beta * population_{t-1} + \gamma * population_{t-1} \quad \textrm{(income/month)} \\
\end{align}
where:

$
\begin{align}
\alpha: \quad &\textrm{'increase_rate'}\\
\epsilon: \quad &\textrm{'decrease_rate'}\\
\beta: \quad &\textrm{'buy_rate'}\\
\gamma: \quad &\textrm{'sell_rate'}\\
\end{align}
$

<center>
<img src="./images/FinTech_SystemDinamics.jpg"
     alt="Diff spec"
     style="width: 100%" />
</center>

In [4]:
def p_new_agent(params, 
                substep, 
                state_history, 
                previous_state):
    p = previous_state['population']
    increase_rate = params['increase_rate']
    t = previous_state['timestep']
    rand = np.random.rand()
    division = t % 31 == 0
    
    if rand < increase_rate and p <= 10000:
        increase_rate = increase_rate
    elif p >= 10000 and division == True:
        increase_rate = random.uniform(0, 0.04)
    else: 
        increase_rate = 0
    value = p * increase_rate
    return {'add_to_agent': value}
    

def p_remove_agent(params, 
                   substep, 
                   state_history, 
                   previous_state):
    p = previous_state['population']
    decrease_rate = params['decrease_rate']
    t = previous_state['timestep']
    division =  t % 31  == 0
    if division == True:
        decrease_rate = decrease_rate
    else: 
        decrease_rate = 0
    value = p * decrease_rate
    return {'add_to_agent': -1.0 * value}

    
def p_sales(params, 
            substep, 
            state_history, 
            previous_state):
    p = float(previous_state['population'])
    buys = params['buy_rate']
    sells = params['sell_rate']
    erMargin = random.uniform(0, 0.15)
    const_fee = params['fees']
    interaction = 0.5
    random_amount_to_buy = random.randint(100, 200)
    random_amount_to_sell = random.randint(100, 200)
    rd = random.uniform(0, 0.25)
    timestep = previous_state['timestep']
    division =  timestep % 2  == 0
    if (interaction > random.random()) & (division == True):
        interacting_buy = (p * (buys-erMargin)) * random_amount_to_buy
    else:
        interacting_buy = 0
    
    if (interaction > np.random.random()) and (division == True):
        interacting_sell = (p * (sells - erMargin)) * random_amount_to_sell
    else:
        interacting_sell = 0
    sales_total = (interacting_buy + interacting_sell) * const_fee
    return{'total_sales': sales_total}
    
    
    
def p_expenses(params, 
               substep, 
               state_history, 
               previous_state):
    p = float(previous_state['population'])
    t = previous_state['treasury']
    timestep = previous_state['timestep']
    division =  timestep % 31  == 0
    kyc = params['expenses_kyc']
    exchange_provider = params['expenses_exchange_provider']
    liquidity_providers = params['expenses_liquidity_providers']
    fiat_gateway = params['expenses_fiat_gateway']
    wallet_provider = params['expenses_wallet_provider']
    if division == True and p > 1000:
        expenses = kyc + wallet_provider + (t * exchange_provider) + (t * fiat_gateway) + (t * liquidity_providers)
    elif division == True and p < 1000:
        kyc_l = 2000
        wallet_provider_l = 2000
        expenses = kyc_l + wallet_provider_l + (t * exchange_provider) + (t * fiat_gateway) + (t * liquidity_providers)
    else:
        expenses = 0
 
    expenses_value = expenses
    return{'total_expenses': expenses_value}

In [5]:
def p_exchange(params, 
               substep, 
               state_history, 
               previous_state):
    p = float(previous_state['population'])
    t = previous_state['treasury']
    timestep = previous_state['timestep']
    division =  timestep % 31  == 0
    exchange_provider = params['expenses_exchange_provider']

    if division == True:
        expenses = exchange_provider * t
    else:
        expenses = 0
    expenses_value = expenses
    return{'total_e_exchange': expenses_value}

In [6]:
def p_kyc(params, 
               substep, 
               state_history, 
               previous_state):
    p = float(previous_state['population'])
    t = previous_state['treasury']
    timestep = previous_state['timestep']
    division =  timestep % 31  == 0
    kyc = params['expenses_kyc']
    if division == True:
        expenses = kyc
    else:
        expenses = 0
    expenses_kyc = expenses
    return{'total_e_kyc': expenses_kyc}

In [7]:
def p_wallet(params, 
               substep, 
               state_history, 
               previous_state):
    p = float(previous_state['population'])
    t = previous_state['treasury']
    timestep = previous_state['timestep']
    division =  timestep % 31  == 0
    wallet_provider = params['expenses_wallet_provider']

    if division == True:
        expenses = wallet_provider
    elif division == True and p > 1000:
        expenses = 2000
    else:
        expenses = 0
    expenses_wallet = expenses
    return{'total_e_wallet': expenses_wallet}

In [8]:
def p_liquididty(params, 
               substep, 
               state_history, 
               previous_state):
    p = float(previous_state['population'])
    t = previous_state['treasury']
    timestep = previous_state['timestep']
    division =  timestep % 31  == 0
    liquidity_providers = params['expenses_liquidity_providers']
    if division == True:
        expenses = t * liquidity_providers
    else:
        expenses = 0
 
    expenses_liquidity = expenses
    return{'total_e_liquidity': expenses_liquidity}

In [9]:
def p_fiat(params, 
               substep, 
               state_history, 
               previous_state):
    p = float(previous_state['population'])
    t = previous_state['treasury']
    timestep = previous_state['timestep']
    division =  timestep % 31  == 0
    fiat_gateway = params['expenses_fiat_gateway']

    if division == True:
        expenses = t * fiat_gateway
    else:
        expenses = 0
 
    expenses_fiat = expenses
    return{'total_e_fiat': expenses_fiat}

## 2. State Update


In [10]:

def s_treasury(params, substep, state_history, previous_state, policy_input):
    treasury = previous_state['treasury'] + policy_input['total_sales']
    return ('treasury', max(treasury, 0))

def s_expenses(params, substep, state_history, previous_state, policy_input):
    expenses = previous_state['expenses'] + policy_input['total_expenses']
    return ('expenses', max(expenses, 0))

def s_population(params, substep, state_history, previous_state, policy_input):
    population = previous_state['population']
    add_to_agent = policy_input['add_to_agent']
    updated_population = population + add_to_agent
    return 'population', max(math.ceil(updated_population), 0)

def s_net_income(params, substep, state_history, previous_state, policy_input):
    net_income = previous_state['treasury'] - previous_state['expenses']
    return ('net_income', max(net_income, 0))
    


In [11]:
def s_kyc(params, substep, state_history, previous_state, policy_input):
    expenses_kyc = previous_state['external_kyc'] + policy_input['total_e_kyc']
    return ('external_kyc', max(expenses_kyc, 0))

In [12]:
def s_exchange(params, substep, state_history, previous_state, policy_input):
    expenses_exchange = previous_state['external_exchange'] + policy_input['total_e_exchange']
    return ('external_exchange', max(expenses_exchange, 0))

In [13]:
def s_wallet(params, substep, state_history, previous_state, policy_input):
    expenses_wallet = previous_state['external_wallet'] + policy_input['total_e_wallet']
    return ('external_wallet', max(expenses_wallet, 0))

In [14]:
def s_liquidity(params, substep, state_history, previous_state, policy_input):
    expenses_liquidity = previous_state['external_liquidity'] + policy_input['total_e_liquidity']
    return ('external_liquidity', max(expenses_liquidity, 0))

In [15]:
def s_fiat(params, substep, state_history, previous_state, policy_input):
    expenses_fiat = previous_state['external_fiat'] + policy_input['total_e_fiat']
    return ('external_fiat', max(expenses_fiat, 0))

## 3. Partial State Update


In [16]:
partial_state_update_blocks = [
    {
        'policies': {
            'total_expenses': p_expenses,
            'total_e_kyc': p_kyc,
            'total_e_exchange': p_exchange,
            'total_e_wallet': p_wallet,
            'total_e_liquididty': p_liquididty,
            'total_e_fiat': p_fiat,
            'sales': p_sales,
            'p_new_agents': p_new_agent,
            'p_remove_agents': p_remove_agent
        },
        'variables': {
            'popilation': s_population,
            'treasury': s_treasury,
            'expenses': s_expenses,
            'net_income': s_net_income,
            'external_kyc': s_kyc,
            'external_exchange': s_exchange,
            'external_liquidity': s_liquidity,
            'external_fiat': s_fiat,
            'external_wallet': s_wallet,
        }
    }
]

## 3. Configuration

In [17]:
MONTE_CARLO_RUNS = 5
SIMULATION_TIMESTEPS = 365

sim_config = config_sim({
    'N': MONTE_CARLO_RUNS,
    'T': range(SIMULATION_TIMESTEPS),
    'M': system_params
})


In [18]:
del configs[:] # Clear any prior configs

In [19]:
experiment = Experiment()
experiment.append_configs(
    initial_state = initial_state,
    partial_state_update_blocks = partial_state_update_blocks,
    sim_configs = sim_config
)

## 4. Execution

In [20]:
exec_mode = ExecutionMode()
exec_context = ExecutionContext()
simulation = Executor(exec_context=exec_context, configs=configs)
raw_result, tensor_field, sessions = simulation.execute()


                  ___________    ____
  ________ __ ___/ / ____/   |  / __ \
 / ___/ __` / __  / /   / /| | / / / /
/ /__/ /_/ / /_/ / /___/ ___ |/ /_/ /
\___/\__,_/\__,_/\____/_/  |_/_____/
by cadCAD

Execution Mode: local_proc
Configuration Count: 2
Dimensions of the first simulation: (Timesteps, Params, Runs, Vars) = (365, 10, 5, 9)
Execution Method: local_simulations
SimIDs   : [0, 0, 0, 0, 0, 1, 1, 1, 1, 1]
SubsetIDs: [0, 0, 0, 0, 0, 1, 1, 1, 1, 1]
Ns       : [0, 1, 2, 3, 4, 0, 1, 2, 3, 4]
ExpIDs   : [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Execution Mode: parallelized
Total execution time: 0.44s


In [21]:
simulation_result = pd.DataFrame(raw_result)
simulation_result

,population,treasury,expenses,net_income,external_kyc,external_exchange,external_liquidity,external_fiat,external_wallet,simulation,subset,run,substep,timestep
0,100,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0,0,1,0,0
1,90,73.053445,4000.000000,0.000000,500.0,0.000000,0.000000,0.000000,1000.0,0,0,1,1,1
2,90,73.053445,4000.000000,0.000000,500.0,0.000000,0.000000,0.000000,1000.0,0,0,1,1,2
3,106,91.642186,4000.000000,0.000000,500.0,0.000000,0.000000,0.000000,1000.0,0,0,1,1,3
4,106,91.642186,4000.000000,0.000000,500.0,0.000000,0.000000,0.000000,1000.0,0,0,1,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3655,10669,139536.793739,36796.513576,100946.642412,6000.0,79.138701,527.591341,13189.783534,12000.0,1,1,5,1,361
3656,10669,139536.793739,36796.513576,102740.280163,6000.0,79.138701,527.591341,13189.783534,12000.0,1,1,5,1,362
3657,10669,141920.750510,36796.513576,102740.280163,6000.0,79.138701,527.591341,13189.783534,12000.0,1,1,5,1,363
3658,10669,141920.750510,36796.513576,105124.236934,6000.0,79.138701,527.591341,13189.783534,12000.0,1,1,5,1,364


## 5. Sweep Data Parameters


In [22]:
# Convert raw results to a Pandas DataFrame
df = pd.DataFrame(raw_result)

# Insert cadCAD parameters for each configuration into DataFrame
for config in configs:
    # Get parameters from configuration
    parameters = config.sim_config['M']
    # Get subset index from configuration
    subset_index = config.subset_id
    
    # For each parameter key value pair
    for (key, value) in parameters.items():
        # Select all DataFrame indices where subset == subset_index
        dataframe_indices = df.eval(f'subset == {subset_index}')
        # Assign each parameter key value pair to the DataFrame for the corresponding subset
        df.loc[dataframe_indices, key] = value

df.head(100)

,population,treasury,expenses,net_income,external_kyc,external_exchange,external_liquidity,external_fiat,external_wallet,simulation,...,increase_rate,decrease_rate,fees,buy_rate,sell_rate,expenses_kyc,expenses_exchange_provider,expenses_liquidity_providers,expenses_fiat_gateway,expenses_wallet_provider
0,100,0.000000,0.000000,0.0,0.0,0.000000,0.00000,0.000000,0.0,0,...,0.17,0.1,0.01,0.25,0.3,500.0,0.00015,0.001,0.025,1000.0
1,90,73.053445,4000.000000,0.0,500.0,0.000000,0.00000,0.000000,1000.0,0,...,0.17,0.1,0.01,0.25,0.3,500.0,0.00015,0.001,0.025,1000.0
2,90,73.053445,4000.000000,0.0,500.0,0.000000,0.00000,0.000000,1000.0,0,...,0.17,0.1,0.01,0.25,0.3,500.0,0.00015,0.001,0.025,1000.0
3,106,91.642186,4000.000000,0.0,500.0,0.000000,0.00000,0.000000,1000.0,0,...,0.17,0.1,0.01,0.25,0.3,500.0,0.00015,0.001,0.025,1000.0
4,106,91.642186,4000.000000,0.0,500.0,0.000000,0.00000,0.000000,1000.0,0,...,0.17,0.1,0.01,0.25,0.3,500.0,0.00015,0.001,0.025,1000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1354,5578.585963,13741.224868,0.0,2000.0,1.383699,9.22466,230.616508,4000.0,0,...,0.17,0.1,0.01,0.25,0.3,500.0,0.00015,0.001,0.025,1000.0
96,1354,5578.585963,13741.224868,0.0,2000.0,1.383699,9.22466,230.616508,4000.0,0,...,0.17,0.1,0.01,0.25,0.3,500.0,0.00015,0.001,0.025,1000.0
97,1354,5578.585963,13741.224868,0.0,2000.0,1.383699,9.22466,230.616508,4000.0,0,...,0.17,0.1,0.01,0.25,0.3,500.0,0.00015,0.001,0.025,1000.0
98,1354,5578.585963,13741.224868,0.0,2000.0,1.383699,9.22466,230.616508,4000.0,0,...,0.17,0.1,0.01,0.25,0.3,500.0,0.00015,0.001,0.025,1000.0


## 6. Analysis

In [23]:
px.line(
    df,
    x='timestep', # Variable on the horizontal axis
    y=['population', 'treasury', 'expenses', 'net_income'], # Variables on the vertical axis
    facet_row='run',
    facet_col='subset',
    title='Users & Income',

    height=800,
)

In [24]:
px.line(
    df,
    x='timestep', # Variable on the horizontal axis
    y=['population', 'treasury', 'external_kyc', 'external_exchange', 'external_liquidity', 'external_fiat', 
'external_wallet', 'expenses'], # Variables on the vertical axis
    facet_row='run',
    facet_col='subset',
    title='Detailed Expenses',
    height=800,
)

\begin{align}
\large population_t &\large= population_{t-1} + {\Delta population} \quad \textrm{(users)} \tag{1} \\
\large treasury_t &\large= treasury_{t-1} + ({\Delta treasury} * fees \quad \textrm{(income from fees)} \tag{2}
\end{align}

where the rate of change ($\Delta$) is:
\begin{align}
\large {\Delta population} &\large= \alpha * population_{t-1} - \epsilon * population_{t-1} \quad \textrm{(new users/month)} \\
\large {\Delta treasury} &\large= \beta * population_{t-1} + \gamma * population_{t-1} \quad \textrm{(income/month)} \\
\end{align}
where:

$
\begin{align}
\alpha: \quad &\textrm{'increase_rate' = 17% and 20%}\\
\epsilon: \quad &\textrm{'decrease_rate' = 0.9% and 1%}\\
\beta: \quad &\textrm{'buy_rate'}\\
\gamma: \quad &\textrm{'sell_rate'}\\
\end{align}
$

In [25]:
pd.options.plotting.backend = "plotly"

In [26]:
px.bar(
    df,
    x='timestep', # Variable on the horizontal axis
    y=['external_kyc', 'expenses'], # Variables on the vertical axis
    facet_row='run',
    facet_col='subset',
    title='Detailed Expenses',
    height=800,
)

In [27]:
# Filter data frame so that we get the data from the 1st MC run
filtered_df = df.query('run == 1')

filtered_df.plot(
    kind='line',
    x='expenses', # Variable on the horizontal axis
    y='population', # Variables on the vertical axis
    color='increase_rate' # Color by prey_death_parameter
)

In [32]:
filtered_df = df.query('decrease_rate == 0.1 & run == 1')

filtered_df.plot(
    kind='line',
    x='population', # Variable on the horizontal axis
    y='treasury', # Variables on the vertical axis
    title="decrease_rate")

In [31]:
filtered_df = df.query('decrease_rate == 0.12 & run == 1')

filtered_df.plot(
    kind='line',
    x='population', # Variable on the horizontal axis
    y='treasury', # Variables on the vertical axis
    title="Decrease rate")